# Step1: Read RawData From Doris

In [44]:
import os
import pymysql
import logging
import pandas as pd
from dotenv import load_dotenv
load_dotenv()

import sys
sys.path.append('Documents/enlighten/analytics/python/tigergraph_profit')
from db_client import DorisClient

mysql_host = os.getenv("mysql_host")
mysql_prot = os.getenv("mysql_prot")
mysql_user = os.getenv("mysql_user")
mysql_pd = os.getenv("mysql_pd")


def get_nft_trade(start_block,end_block):
    sql = """
            select from_address,to_address
                ,transaction_hash as txn_hash
                ,block_number
                ,log_index
                ,trade_eth_value as buyer_pay_amt
                ,seller_receive_amount as seller_receive_amt
                ,token_address
                ,token_ids
                ,token_num
                ,1 as trade_type
                ,trade_time
                ,case when category = 'sell offer' then 1
                      when category = 'buy offer' then 2
                 else 0 end as trade_category
                ,gas_cost
            from dw.dwb_nft_trade_eth_detail_hi
            where block_number > {start_block}
            and block_number < {end_block}

            union all
            select "0x0000000000000000000000000000000000000000" as from_address
                ,account_address as to_address
                ,transaction_hash as txn_hash
                ,block_number
                ,log_index
                ,(gas_cost+mint_cost) as buyer_pay_amt
                ,mint_cost as seller_receive_amt
                ,token_address
                ,token_id as token_ids
                ,mint_amount as token_num
                ,2 as trade_type
                ,trade_time
                ,0 as trade_category
                ,gas_cost
            from dw.dwb_nft_mint_detail_eth_hi
            where block_number > {start_block}
            and block_number < {start_block}
        """.format(
            start_block = start_block,
            end_block = end_block
        )
    doris = DorisClient(mysql_host,mysql_prot,mysql_user,mysql_pd)
    res,fields = doris.read_sql(sql)
    return res,fields

def get_nft_transfer(start_block,end_block):
    sql = """
        select a.from_address,a.to_address
            ,a.transaction_hash as txn_hash
            ,a.block_number
            ,a.log_index
            ,0 as buyer_pay_amt
            ,0 as seller_receive_amt
            ,a.token_address
            ,a.token_id
            ,a.token_num
            ,row_number() over(partition by a.transaction_hash order by a.log_index asc) as index_in_log
            ,0 as trade_type
            ,from_unixtime(b.timestamp) as trade_time
            ,0 as trade_category
            ,(effective_gas_price * t.gas_used/ POWER(10.0, 18)) / count() over(partition by a.transaction_hash) as gas_cost
        FROM  prod.nft_transfer_eth as a
        left join prod.blocks as b
        on a.block_number = b.block_number
        and b.`chain` = 'eth'
        left join prod.transactions_eth t
        on t.block_number > {start_block} and t.block_number < {end_block}
        and t.block_number = a.block_number and t.transaction_hash = a.transaction_hash
        where a.block_number > {start_block}
        and a.block_number < {end_block}
    """.format(
        start_block = start_block,
        end_block = end_block
    )
    doris = DorisClient(mysql_host,mysql_prot,mysql_user,mysql_pd)
    res,fields = doris.read_sql(sql)
    return res,fields

res,fields = get_nft_trade(15400000,15400500)
fields

['from_address',
 'to_address',
 'txn_hash',
 'block_number',
 'log_index',
 'buyer_pay_amt',
 'seller_receive_amt',
 'token_address',
 'token_ids',
 'token_num',
 'trade_type']

# Step2 TigerGraph Client Setup

In [14]:
import json
import os
import requests
from requests.auth import HTTPBasicAuth
from typing import List
from loguru import logger

from dotenv import load_dotenv 
load_dotenv()

import sys
sys.path.append('Documents/enlighten/analytics/python/tigergraph_profit')
from db_client import TgGraphClient

tg_server_ip = os.getenv('tg_server_ip')
tg_graph_name = 'NFT_Profit_Transfer'
tg_username = os.getenv('tg_username')
tg_password = os.getenv('tg_password')
url = 'http://%s:9000/graph/%s' % (tg_server_ip, tg_graph_name)


def get_trade_primary_id(row,field_names):
    # get the index of sql output fields
    block_number_index = field_names.index("block_number")
    log_index = field_names.index("log_index")
    token_ids_index = field_names.index("token_ids")
    token_num_index = field_names.index("token_num")
    buyer_pay_index = field_names.index("buyer_pay_amt")
    seller_receive_index = field_names.index("seller_receive_amt")

    # update values for bundle sales cases
    row = list(row)
    token_id_cnt = len(row[token_ids_index].split(","))
    if token_id_cnt > 1:
        row[buyer_pay_index] = row[buyer_pay_index]/token_id_cnt
        row[seller_receive_index] = row[seller_receive_index]/token_id_cnt
        row[token_num_index] = row[token_num_index]/token_id_cnt

    # bundles sales:one line to multi lines with primiary key
    trade_rows = []
    for index_in_log in range(token_id_cnt):
        primary_id = "_".join([
            str(row[block_number_index]),
            str(row[log_index]),
            str(index_in_log)
        ])
        tmp_row = row + [primary_id]
        trade_rows.append(tmp_row)
    return trade_rows

def get_vertex_entities(formated_row,field_names):
    # vertex input format
    vertex_dict = {'nft_transfer':{}}
    vertex_value_dict = {}
    for field in field_names:
        field_index = field_names.index(field)
        if field == 'token_ids':
            token_ids = formated_row[field_index]
            index_in_log = int(formated_row[-1].split("_")[2])
            token_id_value = token_ids.split(",")[index_in_log] if len(token_ids.split(",")) > 1 else token_ids
            vertex_value_dict['token_id'] = {
                'value':token_id_value
            }
        else:
            vertex_value_dict[field] = {
                'value':formated_row[field_index]
            }
    vertex_dict['nft_transfer'][formated_row[-1]] = vertex_value_dict
    return vertex_dict

def get_edge_entities(formated_row,field_names):
    # edge input format
    edge_dict = {}
    from_address_index = field_names.index("from_address")
    to_address_index = field_names.index("to_address")
    from_address = formated_row[from_address_index]
    to_address = formated_row[to_address_index]
    trade_primary_id = formated_row[-1]
    print(from_address,to_address,trade_primary_id)
    edge_dict["account"] = {from_address:{"send_nft":{"nft_transfer":{trade_primary_id:{}}}}}
    return edge_dict
    
def insert(entities):
    url = 'http://%s:9000/graph/%s' % (tg_server_ip, graph_name)
    print(url)
    result = requests.post(url, json=entities, auth=HTTPBasicAuth(username,password))
    return result

In [12]:
get_trade_primary_id(res[1],fields)[0]

['0x154b0c6f694db030225dde4a667f5a6a69a32e5f',
 '0xfefef682c54d7dc0b7bede523db0a409729be2df',
 '0xebac1559f3a3b47d5462519f2cb4a8b631bac195b3338c983b8a1a7ee9f94f25',
 15449728,
 259,
 0.015,
 0.014624999999999999,
 '0x4f70d1c9d5c8a57db5acc139915f69d06bd22f49',
 '3697,3698,3699',
 1.0,
 1,
 '15449728_259_0']

In [15]:
get_edge_entities(get_trade_primary_id(res[1],fields)[0],fields)

0x154b0c6f694db030225dde4a667f5a6a69a32e5f 0xfefef682c54d7dc0b7bede523db0a409729be2df 15449728_259_0


{'account': {'0x154b0c6f694db030225dde4a667f5a6a69a32e5f': {'send_nft': {'nft_transfer': {'15449728_259_0': {}}}}}}

# Testing

In [19]:
from dotenv import load_dotenv 
load_dotenv()
tg_username = os.getenv("tg_username")
tg_password = os.getenv("tg_password")

entities_dict = {}
entities_dict['vertices'] = get_vertex_entities(get_trade_primary_id(res[1],fields)[0],fields)
entities_str = json.dumps(entities_dict)
entities = json.loads(entities_str)
print(entities)

result = requests.post(url, json=entities, auth=HTTPBasicAuth(tg_username,tg_password))
data = json.loads(result.text, strict=False)
# check_and_parse(data)
print(data)

{'vertices': {'nft_transfer': {'15400581_457_0': {'from_address': {'value': '0x172870406a9ce670c6b0921f48ff6105742a65a7'}, 'to_address': {'value': '0xd70650cc0e0f9238bcccb14f44db775f9740c9b9'}, 'txn_hash': {'value': '0x43cb328e3fafda603c52123eb14bad9b04ed62fa4c58a0f97960b634c668d068'}, 'block_number': {'value': 15400581}, 'log_index': {'value': 457}, 'buyer_pay_amt': {'value': 0.24}, 'seller_receive_amt': {'value': 0.21}, 'token_address': {'value': '0x8b3c69abc8c588e3eceb2eedf3bccf8fea950241'}, 'token_id': {'value': '392'}, 'token_num': {'value': 1.0}, 'trade_type': {'value': 1}}}}}
{'version': {'edition': 'enterprise', 'api': 'v2', 'schema': 27}, 'error': False, 'message': '', 'results': [{'accepted_vertices': 1, 'accepted_edges': 0}], 'code': 'REST-0001'}


In [19]:
from dotenv import load_dotenv 
load_dotenv()
tg_username = os.getenv("tg_username")
tg_password = os.getenv("tg_password")

edge_dict = {}
edge_dict['edges'] = get_edge_entities(get_trade_primary_id(res[1],fields)[0],fields)
edge_entities_str = json.dumps(edge_dict)
entities = json.loads(edge_entities_str)
print(entities)

result = requests.post(url, json=entities, auth=HTTPBasicAuth(tg_username,tg_password))
data = json.loads(result.text, strict=False)
print(data)

0x154b0c6f694db030225dde4a667f5a6a69a32e5f 0xfefef682c54d7dc0b7bede523db0a409729be2df 15449728_259_0
{'edges': {'account': {'0x154b0c6f694db030225dde4a667f5a6a69a32e5f': {'send_nft': {'nft_transfer': {'15449728_259_0': {}}}}}}}
{'version': {'edition': 'enterprise', 'api': 'v2', 'schema': 27}, 'error': False, 'message': '', 'results': [{'accepted_vertices': 0, 'accepted_edges': 1}], 'code': 'REST-0002'}


In [32]:
def check_and_parse(result: requests.Response):
    if result.status_code != 200:
        logger.error("code: {}, body: {}", result.status_code, result.text)
        raise Exception(result.status_code)
    data = json.loads(result.text, strict=False)
    x = data.get('error', False)
    if x == 'false':
        x = False
    if x:
        raise Exception(result.text)
    return data

check_and_parse

{'version': {'edition': 'enterprise', 'api': 'v2', 'schema': 27},
 'error': False,
 'message': '',
 'results': [{'accepted_vertices': 1, 'accepted_edges': 0}],
 'code': 'REST-0001'}

# Main

In [31]:
import json

def main():
    res = get_nft_trade(15400000,15450000)
    print(res)
    
if __name__ == "__main__":
    main()

(('0x616ed054e0e0fdbfcad3fa2f42daed3d7d4ee448', '0xe76c2e2f45eec562e029d9f139a906a0259ec39d', '0xe4556462c1e1696f4d3ff3e1ea941b9899c5f11abe1dc78ca723c8b7f0d76cb3', 15400006, 189, 0.028, 0.02534, '0x39223e2596bf8e1de3894f66947cacc614c24a2f', '2746', 1, 1), ('0x616ed054e0e0fdbfcad3fa2f42daed3d7d4ee448', '0xe76c2e2f45eec562e029d9f139a906a0259ec39d', '0xe4556462c1e1696f4d3ff3e1ea941b9899c5f11abe1dc78ca723c8b7f0d76cb3', 15400006, 190, 0.028, 0.02534, '0x39223e2596bf8e1de3894f66947cacc614c24a2f', '2732', 1, 1), ('0x616ed054e0e0fdbfcad3fa2f42daed3d7d4ee448', '0xe76c2e2f45eec562e029d9f139a906a0259ec39d', '0xe4556462c1e1696f4d3ff3e1ea941b9899c5f11abe1dc78ca723c8b7f0d76cb3', 15400006, 191, 0.028, 0.02534, '0x39223e2596bf8e1de3894f66947cacc614c24a2f', '2729', 1, 1), ('0xd9bc1fc51b8d8369980505aa96e975da03346b4a', '0xe76c2e2f45eec562e029d9f139a906a0259ec39d', '0xe4556462c1e1696f4d3ff3e1ea941b9899c5f11abe1dc78ca723c8b7f0d76cb3', 15400006, 192, 0.029, 0.026245, '0x39223e2596bf8e1de3894f66947cacc614c2

In [46]:
def get_compound_users():
    connection = pymysql.connect(host=mysql_host, user=mysql_user, password=mysql_pd, db='dw', port=int(mysql_prot), charset='utf8')
    cursor = connection.cursor()

    # get max block_number
    max_blo_sql = """
        select max(block_number) as block_number
        from dw.dws_token_balance_eth"""
    cursor.execute(max_blo_sql)
    max_blo = cursor.fetchall()[0][0]

    
    sql = """
        select account_address
        from dw.dws_token_balance_eth
        where token_address in (
            '0x041171993284df560249b57358f931d9eb7b925d',
            '0x12392f67bdf24fae0af363c24ac620a2f67dad86',
            '0x158079ee67fce2f58472a96584a73c7ab9ac95c1',
            '0x35a18000230da775cac24873d00ff85bccded550',
            '0x39aa39c021dfbae8fac545936693ac917d5e7563',
            '0x4b0181102a0112a2ef11abee5563bb4a3176c9d7',
            '0x4ddc2d193948926d02f9b1fe9e1daa0718270ed5',
            '0x5d3a536e4d6dbd6114cc1ead35777bab948e3643',
            '0x6c8c6b02e7b2be14d4fa6022dfd6d75921d90e4e',
            '0x70e36f6bf80a52b3b46b3af8e106cc0ed743e8e4',
            '0x7713dd9ca933848f6819f38b8352d9a15ea73f67',
            '0x80a2ae356fc9ef4305676f7a3e2ed04e12c33946',
            '0x95b4ef2869ebd94beb4eee400a99824bf5dc325b',
            '0xb3319f5d18bc0d84dd1b4825dcde5d5f7266d407',
            '0xc11b1268c1a384e55c48c2391d8d480264a3a7f4',
            '0xccf4429db6322d5c611ee964527d42e5d685dd6a',
            '0xe65cdb6479bac1e22340e4e755fae7e509ecd06c',
            '0xf5dce57282a584d2746faf1593d3121fcac444dc',
            '0xf650c3d88d12db855b8bf7d11be6c55a4e07dcc9',
            '0xface851a4921ce59e912d19329929ce6da6eb0c7'
        )
        group by 1
    """
    cursor.execute(sql)
    res_rows = cursor.fetchall()
    return res_rows,max_blo

res,max_blo = get_compound_users()
print(len(res),max_blo)

401744 15582482
